## Normalization

In [1]:
import h5py
import torch
import numpy as np
import matplotlib.pyplot as plt

# with h5py.File(f'data_large/Burgers_train_100000_default.h5', 'r') as f:
#     # Traj_dataset.traj_train = torch.tensor(f['train']['pde_140-256'][:10000, :131], dtype=torch.float32, device=cfg.device)
#     traj = torch.tensor(f['train']['pde_140-256'][:1000, :131], dtype=torch.float32)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
import hydra
from omegaconf import DictConfig, OmegaConf
from tqdm import tqdm

hydra.initialize(config_path="cfg_time_batch", version_base=None)
cfg = hydra.compose(config_name="config", overrides=["task=KdV", "nt=131", "initial_datasize=64", "model.n_modes=[256]"])

from utils import set_seed
set_seed(100)

In [3]:
def get_gradient_norm(model):
    total_norm = 0
    for p in model.parameters():
        if p.grad is not None:
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item() ** 2
    return total_norm ** 0.5

In [4]:
with h5py.File(cfg.dataset.train_path, 'r') as f:
    # Traj_dataset.traj_train = torch.tensor(f['train']['pde_140-256'][:10000, :131], dtype=torch.float32, device=cfg.device)
    traj = torch.tensor(f['train']['pde_140-256'][:1000, :131], dtype=torch.float32)

mean = 0
std = 1
print(f'Mean: {mean}, Std: {std}')
traj = (traj - mean) / std

from neuralop.models import FNO

unrolling = cfg.train.unrolling
nt = cfg.nt
ensemble_size = cfg.ensemble_size
num_acquire = cfg.num_acquire
device = cfg.device
epochs = cfg.train.epochs
lr = cfg.train.lr
batch_size = cfg.train.batch_size
initial_datasize = cfg.initial_datasize



def train(Y, train_nts, **kwargs):
    model = FNO(n_modes=cfg.model.n_modes, hidden_channels=64,
                in_channels=1, out_channels=1)

    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
    criterion = torch.nn.MSELoss()

    inputs = []
    outputs = []
    for b in range(Y.shape[0]):
        for t in range(train_nts[b].item()-1):
            inputs.append(Y[b,t])
            outputs.append(Y[b, t+1])
    inputs = torch.stack(inputs, dim=0).unsqueeze(1)
    outputs = torch.stack(outputs, dim=0).unsqueeze(1)

    dataset = torch.utils.data.TensorDataset(inputs, outputs)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model.train()
    for epoch in tqdm(range(epochs)):
        model.train()
        # max_unrolling = epoch if epoch <= unrolling else unrolling
        # unrolling_list = [r for r in range(max_unrolling + 1)]

        total_loss = 0
        max_grad_norm = 0
        for x, y in dataloader:
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            
            pred = model(x)
            loss = criterion(pred, y)

            # loss = torch.sqrt(loss)
            loss.backward()

            current_grad_norm = get_gradient_norm(model)
            max_grad_norm = max(max_grad_norm, current_grad_norm)

            optimizer.step()
            total_loss += loss.item()
        print(f'Max grad norm: {max_grad_norm}')
        scheduler.step()
        # wandb.log({f'train/loss_{acquire_step}': total_loss})
    return model

timestep = (traj.shape[1] - 1) // (nt - 1) # 10
Y = traj[:,0::timestep]
train_nts = torch.ones(Y.shape[0], device=device, dtype=torch.int64)
train_nts[:initial_datasize] = nt

model = train(Y, train_nts)

from utils import torch_expand
from eval_utils import compute_metrics

test_Y = traj[initial_datasize:1000, 0::timestep]

model.eval()

preds = []
preds.append(test_Y[:,0:1])

with torch.no_grad():
    for t in range(nt-1):
        X = preds[-1].to(device)
        pred = model(X) # batch x 1 x 256
        preds.append(pred.cpu())

preds = torch.cat(preds, dim=1) # batch x nt x 256


metrics = compute_metrics(test_Y, preds, d=2, device=device, reduction=True)

print(metrics)

for scale in [1.0]:
    print(scale)

    with h5py.File(cfg.dataset.train_path, 'r') as f:
        # Traj_dataset.traj_train = torch.tensor(f['train']['pde_140-256'][:10000, :131], dtype=torch.float32, device=cfg.device)
        traj = torch.tensor(f['train']['pde_140-256'][:1000, :131], dtype=torch.float32)

    mean = traj[:32].mean()
    std = traj[:32].std()
    print(f'Mean: {mean}, Std: {std}')
    traj = (traj - mean) / std * scale

    from neuralop.models import FNO

    unrolling = cfg.train.unrolling
    nt = cfg.nt
    ensemble_size = cfg.ensemble_size
    num_acquire = cfg.num_acquire
    device = cfg.device
    epochs = cfg.train.epochs
    lr = cfg.train.lr
    batch_size = cfg.train.batch_size
    initial_datasize = cfg.initial_datasize


    def train(Y, train_nts, **kwargs):
        model = FNO(n_modes=cfg.model.n_modes, hidden_channels=64,
                    in_channels=1, out_channels=1)

        model = model.to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
        criterion = torch.nn.MSELoss()

        inputs = []
        outputs = []
        for b in range(Y.shape[0]):
            for t in range(train_nts[b].item()-1):
                inputs.append(Y[b,t])
                outputs.append(Y[b, t+1])
        inputs = torch.stack(inputs, dim=0).unsqueeze(1)
        outputs = torch.stack(outputs, dim=0).unsqueeze(1)

        dataset = torch.utils.data.TensorDataset(inputs, outputs)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

        model.train()
        for epoch in tqdm(range(epochs)):
            model.train()
            # max_unrolling = epoch if epoch <= unrolling else unrolling
            # unrolling_list = [r for r in range(max_unrolling + 1)]

            max_grad_norm = 0
            total_loss = 0
            for x, y in dataloader:
                optimizer.zero_grad()
                x, y = x.to(device), y.to(device)
                
                pred = model(x)
                loss = criterion(pred, y)

                # loss = torch.sqrt(loss)
                loss.backward()

                current_grad_norm = get_gradient_norm(model)
                max_grad_norm = max(max_grad_norm, current_grad_norm)

                optimizer.step()
                total_loss += loss.item()
            print(f'Max grad norm: {max_grad_norm}')
            scheduler.step()
            # wandb.log({f'train/loss_{acquire_step}': total_loss})
        return model
    
    timestep = (traj.shape[1] - 1) // (nt - 1) # 10
    Y = traj[:,0::timestep]
    train_nts = torch.ones(Y.shape[0], device=device, dtype=torch.int64)
    train_nts[:initial_datasize] = nt

    model = train(Y, train_nts)

    from utils import torch_expand
    from eval_utils import compute_metrics

    test_Y = traj[initial_datasize:1000, 0::timestep]

    model.eval()

    preds = []
    preds.append(test_Y[:,0:1])

    with torch.no_grad():
        for t in range(nt-1):
            X = preds[-1].to(device)
            pred = model(X) # batch x 1 x 256
            preds.append(pred.cpu())

    preds = torch.cat(preds, dim=1) # batch x nt x 256


    metrics = compute_metrics(test_Y, preds, d=2, device=device, reduction=True)

    print(metrics)

# print('max-min scaling')

# with h5py.File(cfg.dataset.train_path, 'r') as f:
#     # Traj_dataset.traj_train = torch.tensor(f['train']['pde_140-256'][:10000, :131], dtype=torch.float32, device=cfg.device)
#     traj = torch.tensor(f['train']['pde_140-256'][:1000, :131], dtype=torch.float32)

# max = traj.max()
# min = traj.min()
# print(f'Max: {max}, Min: {min}')
# traj = (traj - (max+min/2)) / (max - min)

# from neuralop.models import FNO

# unrolling = cfg.train.unrolling
# nt = cfg.nt
# ensemble_size = cfg.ensemble_size
# num_acquire = cfg.num_acquire
# device = cfg.device
# epochs = cfg.train.epochs
# lr = cfg.train.lr
# batch_size = cfg.train.batch_size
# initial_datasize = cfg.initial_datasize

# def train(Y, train_nts, **kwargs):
#     model = FNO(n_modes=cfg.model.n_modes, hidden_channels=64,
#                 in_channels=1, out_channels=1)

#     model = model.to(device)

#     optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
#     criterion = torch.nn.MSELoss()

#     inputs = []
#     outputs = []
#     for b in range(Y.shape[0]):
#         for t in range(train_nts[b].item()-1):
#             inputs.append(Y[b,t])
#             outputs.append(Y[b, t+1])
#     inputs = torch.stack(inputs, dim=0).unsqueeze(1)
#     outputs = torch.stack(outputs, dim=0).unsqueeze(1)

#     dataset = torch.utils.data.TensorDataset(inputs, outputs)
#     dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

#     model.train()
#     for epoch in tqdm(range(epochs)):
#         model.train()
#         # max_unrolling = epoch if epoch <= unrolling else unrolling
#         # unrolling_list = [r for r in range(max_unrolling + 1)]

#         total_loss = 0
#         for x, y in dataloader:
#             optimizer.zero_grad()
#             x, y = x.to(device), y.to(device)
            
#             pred = model(x)
#             loss = criterion(pred, y)

#             # loss = torch.sqrt(loss)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()
#         scheduler.step()
#         # wandb.log({f'train/loss_{acquire_step}': total_loss})
#     return model

# timestep = (traj.shape[1] - 1) // (nt - 1) # 10
# Y = traj[:,0::timestep]
# train_nts = torch.ones(Y.shape[0], device=device, dtype=torch.int64)
# train_nts[:initial_datasize] = nt

# model = train(Y, train_nts)

# from utils import torch_expand
# from eval_utils import compute_metrics

# test_Y = traj[initial_datasize:1000, 0::timestep]

# model.eval()

# preds = []
# preds.append(test_Y[:,0:1])

# with torch.no_grad():
#     for t in range(nt-1):
#         X = preds[-1].to(device)
#         pred = model(X) # batch x 1 x 256
#         preds.append(pred.cpu())

# preds = torch.cat(preds, dim=1) # batch x nt x 256


# metrics = compute_metrics(test_Y, preds, d=2, device=device, reduction=True)

# print(metrics)

Mean: 0, Std: 1


  1%|          | 1/100 [00:08<14:18,  8.67s/it]

Max grad norm: 1.9372691859292854


  2%|▏         | 2/100 [00:13<10:38,  6.51s/it]

Max grad norm: 0.01978146744480674


...

 99%|█████████▉| 99/100 [10:03<00:05,  5.56s/it]

Max grad norm: 0.0009245817269142255


100%|██████████| 100/100 [10:08<00:00,  6.08s/it]

Max grad norm: 0.0006725323826233204


(tensor(0.3570), tensor(0.2066), tensor(7039.2114))
